# Analytical Dataset (ADS)
The core concept of ADS is that we merge different data sources to have all possible information about objects of interest (the most often, these are our clients) in one place. We will then create a sliding window with all possible metrics.
1. The core part of ADS is a sliding window for each time period, for example, 1 week.
2. ADS contains one line for each observation every week

<img src='images/ADS_timeline.png'>

As indicated in the picture above we use the same history to create each slice, for example 12 months.

We can summarize the main advantages of ADS as
* Combines all data sources into one table
* In the future, ML models can be based on one table
* By using time slices (weekly, monthly) we take care of fluctuations in the data
* It suits as aggregation layer for the reporting
* Batch scoring (weekly, monthly) is easy to implement
* New data sources can be simply added in the future using joins

<img src='images/ADS.png'>

# Tutorial
[**Northwind Database**](https://drive.google.com/file/d/1HCfNF5BsYUrQhhr-vnO_LFgs5THJUAh_/view)

### Best if worked in VSCode following the instructions descrived here:

We are going to build ADS about our customers. We will store information about orders from the last month. We want, in the ADS, to have only ONE line per every customer and every month.

Let's see how many orders we have:
```sql
select count(*) from orders;
select min(orderdate) from orders;
select max(orderdate) from orders;
```
We can see 830 orders ranging from `1996-07-04` to `1998-05-06`. We are going to build ADS with values aggregated by month. We could use the same logic for a week or days but for training purposes, monthly aggregations are sufficient.

In practice, it depends on the industry that required granularity. For traditional banking, 1 month may be enough. For telecommunications, 1 week can be appropriate, but there are also industries like e-commerce where they need to aggregate per day.

For our tutorial, we will aggregate orders from each month and label it with column called `end_obs_date` (end observation date). `end_obs_date` is the column that labels monthly slice we are aggregating to. If anything happens, for example in August 1996, we will assign `end_obs_date` 1st of September, 1996.

For example:
* order date is 1996-12-12 --> endobsdate will be 1997-01-01
* order date is 1997-01-31 --> endobsdate will be 1997-02-01

Therefore, in our ADS, we will have following end observation dates: from `1996-08-01` to `1998-06-01`.

We start with the create of table endobsdates. We will store all possible dates there.
```sql
DROP TABLE if exists end_obs_dates;
CREATE TABLE end_obs_dates 
AS

-- Recursive query example (something like FOR loop for SQL)
WITH RECURSIVE
  cnt(x) AS (
     SELECT 0
     UNION ALL
     SELECT x+1 FROM cnt
      LIMIT (SELECT ROUND(((julianday('1998-06-01') - julianday('1996-08-01'))/30) + 1)) -- this counts number of months between these two days
  )
SELECT date('1996-08-01', '+' || x || ' month') as end_obs_date FROM cnt;
```
    It's good practice to put drop table if exists before each table creation. This will eliminate errors when rerunning the script and doesn't do anything if the table doesn't exist yet.

Now, we create ADS POPULATION HIST(each customer id for each month). We will use table customers for this.
```sql
DROP TABLE if exists ads_population_hist;

CREATE TABLE ads_population_hist 
AS
SELECT A.*,
       B.*
FROM end_obs_dates AS A
CROSS JOIN (SELECT DISTINCT customerid FROM customers) AS B
;
```
Let's see how our new table looks like:
```sql
select * from ads_population_hist limit 10;
```
Now, we have the main table where every customer has exactly one row for every month, even when they didn't buy anything.

Our main goal is to create a table where we keep all the important information about our clients. In this tutorial, we will work with clients' orders and create following variables:
* noofitems
* noofdistinct_orders
* total_price

All of these information will be aggregated on monthly level. We can take this information from the table `Order Details`.

As the first step, we will compute additional attribute `totalprice_for_product` as `unitprice * quantity` using the following select statement.
```sql
SELECT *,
       unitprice*quantity AS totalprice_for_product
FROM "Order Details" LIMIT 20
It is important that in our summary, every order_id is only once.

SELECT A.orderid,
     COUNT(DISTINCT A.productid) AS no_of_distinct_products,
     SUM(A.quantity) AS no_of_items,
     SUM(A.totalprice_for_product) AS total_price 
     FROM (SELECT *,
                  unitprice*quantity AS totalprice_for_product
           FROM "Order Details") AS A
GROUP BY 1;
```
We have aggregated information for every `orderid`. Now, we need to assign it to the correct customer and correct month. we will use table `orders` for that. We can see the sample of that table below.
```sql
SELECT 
    orderid,
    customerid,
    orderdate
FROM orders 
LIMIT 100;
```
We need to assign `end_obs_date` to each order.
```sql
SELECT orderid,
       customerid,
       orderdate,
       date(orderdate,'start of month','+1 month') as end_obs_date
FROM orders LIMIT 100;
```
Now, we will combine previous steps into 1 table called `ads_orders_hist`.
```sql
DROP TABLE if exists ads_orders_hist;

CREATE TABLE ads_orders_hist 
AS
SELECT A.orderid,
       A.customerid,
       A.end_obs_date,
       B.no_of_distinct_products,
       B.no_of_items,
       B.total_price
FROM (
    SELECT orderid,
             customerid,
             orderdate,
             date(orderdate,'start of month','+1 month') as end_obs_date
    FROM orders)
AS A
LEFT OUTER JOIN (
    SELECT A.orderid,
         COUNT(DISTINCT A.productid) AS no_of_distinct_products,
         SUM(A.quantity) AS no_of_items,
         SUM(A.totalprice_for_product) AS total_price
    FROM (
        SELECT *,
            unitprice*quantity AS totalprice_for_product
        FROM "Order Details") 
    AS A
    GROUP BY 1) 
AS B 
ON A.orderid = B.orderid;

;
```
It is important to test that every `orderid` is only **once** in the table.
```sql
select orderid
    ,count(*)
from ads_orders_hist
group by 1
order by 2 desc
limit 5
;
```
If the first line is 1 then we are good to go to the next step.

The last step is to join `ads_orders_hist` with `ads_pop_hist`. We will join everything on columns `customerid` and `end_obs_date`. We will use left join because we want to keep also track of the clients who didn't buy anything in the specific month.
```sql
drop table if exists ads_observation_hist;
create table ads_observation_hist as
select 
    A.*
    -- we can replace missings with 0 because it means there were no orders for this client during specific month.
    ,coalesce(B.no_of_distinct_orders_1M, 0) as no_of_distinct_orders_1M
    ,coalesce(B.no_of_items_1M, 0) as no_of_items_1M
    ,coalesce(B.total_price_1M, 0) as total_price_1M
from ads_population_hist as A
left outer join (
    -- we need to group by our orders to customer level
    select customerid
        ,end_obs_date
        ,count(distinct orderid) as no_of_distinct_orders_1M
        ,sum(no_of_items) as no_of_items_1M
        ,sum(total_price) as total_price_1M
    from ads_orders_hist
    group by 1,2
) as B
on A.customerid = B.customerid
  and A.end_obs_date = B.end_obs_date
;
```
It is important to test again if every `customerid` is in the table only once for every month.
```sql
select customerid
    ,end_obs_date
    ,count(*)
from ads_observation_hist
group by 1,2
order by 3 desc
limit 5
;
```
Let's see our final table:
```sql
select * from ads_observation_hist limit 10;
```
We can create many more variables that could be useful for modeling and further analysis, for example, `max_price`, `min_price`, `avg_price` etc.

Once new data from the next month come in, we can simply add them to these tables using where clause for `orderdate` and `INSERT INTO` statements.

## Conclusion
We have finished our first ADS with a couple of columns. We have named these columns with `_1M` at the end to indicate that these are monthly aggregations.

# Challenge
We'll use the one we have created during the tutorial and we will add additional attributes aggregated for 1 month. Plus, we will use window functions to aggregate orders from 3 and 6 months.

We are going to continue with the creation of the ADS from the tutorial today. So far we have 3 attributes:
* `noofdistinctorders1M`
* `noofitems_1M`
* `totalprice1M`

We want to create additional attributes for every client and every month.

Create new attributes in table ads_observation_hist using VSCode and database Northwind.
* `no_of_items_3M` (use window functions)
* `total_price_3M` (use window functions)
* `max_monthly_total_price_3M` (use window functions)
* `min_monthly_total_price_3M` (use window functions)
* `avg_no_of_items_3M` (use window functions)

All attributes should be in the table that is monthly aggregated and every client is there for every month.

    You can continue directly after the ADS tutorial and build on top of the table `ads_observation_hist`.